In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

OBJETIVO:

En este Nootbook se mostrarán algunas consideraciones relacionadas con los datos obtenidos de de la base datos Bikeshare que contienen los datos de renta de bicicletas por hora para la ciudad de Washington por hora durante 2 años. 

Esta base de datos tiene información sobre las características del momento en el que se rentan las bicicletas tales como: Estación, temperatura, humendad, velocidad del tiempo, variables nominales relacionadas a si el dia en un día festivo o día laboral entre otras.  


In [ ]:

import numpy as np                # linear algebra
import pandas as pd               # data frames
import seaborn as sns             # visualizations
import matplotlib.pyplot as plt   # visualizations
from pylab import *
import scipy.stats                # statistics
from sklearn import preprocessing
import dateutil.parser
from datetime import *



df = pd.read_csv("../input/bikeshare.csv")
# , index_col = 'datetime', parse_dates=True)

print(df.head())
print(df.tail())
print(df.columns.tolist())
print(df.info())
print(df.shape)


En el bloque de arriba importamos las librerias que pueden ser útililes para mostar lo que queremos. Además mostramos unos resúmenes de la base de datos y con ello tener un primer acercamiento con la información, no hay datos nulos y también sabemos el tipo de datos que contiene cada columna. 

Como primer medida se muestran para los dos años en una gráfica el número de rentas de bicicletas

In [ ]:

#código para obtener las rentas contìnuas
sns.lineplot(x="datetime", y="count", data=df)
# plt.ylim(0, 1500)




In [ ]:
nc = (pd.DataFrame(df.groupby('season')['count'].sum()) )
nc.reset_index(inplace=True)
#print(nc)
sns.barplot(x=nc["season"],y=nc["count"], palette=("Blues_d"))

[](http://)Con la gráfica anterior obtenemos el total de rentas por estación teniendo en cuenta que los números en el eje X sigifican lo siguiente.
1: Primavera
2: Verano
3: Otoño
4: Invierno, donde se ven cosas interesantes, como que en iniverno se rentan más bicicletas que en primavera y que relativamnte la rentas en invierno no están lejos de las rentas en verano u otoño. 

In [ ]:
yk = sns.distplot( df["count"])
plt.show()

El anterior es el histograma del número de rentas de biciceltas por hora, la distribución fisher es similar al resultado obtenido.

Por otro lado dos variables que pueden tener relación con la renta de bicicletas es la sensación térmica ("atemp"), al graficar una dispersión entre esta variable y el número de rentas ("count") vemos que aunque aparentemente hay una relación positiva dicha relación no es muy clara.  

In [ ]:
ax = sns.scatterplot(x="atemp", y="count", data=df)

Con el fin de entender un poco mas las relaciones que hay entre las variables, se muestra una tabla de correlaciones,
primero no hay relaciones muy marcadas entre las variables, pero si se pueden ver algunas cosas como una relación inversa
entre humedad y velocidad del viento o la relación positiva entre el número de rentas y temperatura que es la relación que
se veía anteriormente. 

In [ ]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
hd = df[df.holiday == 1]
wd = df[df.workingday == 1]
wend = df[(df.holiday == 0) & (df.workingday == 0)]
wend.head()
wd.head()

average(hd['count'])
average(wd['count'])
average(wend['count'])

                     
h = [average(hd['count']),average(wend['count']),average(wd['count'])]
b = ('hd','wd','wend')
    
y = np.arange(len(b))

plt.bar(y,h)
plt.xticks(y,b)


En el bloque y gráfica anterior obtenemos unos sub dataframes, para el caso de días festivos (hd), días laborales (wd) y fines de semana no festivos (wend) que es cuando las variables nominales están al mismo tiempo en 0, obtenemos un promedio para cada tipo de día, y nos damos cuenta de algo revelador, los fines de semana y días laborales tienen un promedio de rentas que los días festivos.  

In [ ]:
hd['datetime'] = pd.to_datetime(hd['datetime'], errors='coerce')
wd['datetime'] = pd.to_datetime(wd['datetime'], errors='coerce')
wend['datetime'] = pd.to_datetime(wend['datetime'], errors='coerce')

hd['hour'] = hd['datetime'].dt.hour
wd['hour'] = wd['datetime'].dt.hour
wend['hour'] = wend['datetime'].dt.hour

wend.head()


Se agregó una nueva variable que es la hora del día va de la hora 0 a hora 23, para ello fue necesario cambiar el tipo de dato de la columna datetime, antes era tipo obejeto y ahora quedó tipo datetime y así fue posible obtener la hora del día. 

In [ ]:
nc4 = pd.DataFrame((hd.groupby('hour')['count'].mean()) )
nc5 = pd.DataFrame((wd.groupby('hour')['count'].mean()) )
nc6 = pd.DataFrame((wend.groupby('hour')['count'].mean()) )
nc4['day_type'] = repeat('hd',len(nc4['count']))
nc5['day_type'] = repeat('wd',len(nc5['count']))
nc6['day_type'] = repeat('wend',len(nc6['count']))

print(nc4)

df4 = pd.concat([nc4,  nc5, nc6])
df4 = df4.reset_index()
df4.head()



Por último graficaremos los promedios de rentas de bicicletas por tipo de día por cada hora, y así tener un panorama mas claro del comportamiento, graficamos los promedios por hora en barras para los tres tipos de día (hd, wd, wend). Para lo cual fue necesario unir verticalemte los tres dataframes auxiliares que obtenian los promedios y así poder agrupar. 

In [ ]:
#Gráfico de barras promedio rentas por tipo de día. 
g = sns.barplot(x="hour", y="count", hue="day_type", data=df4)
        

Como conclusión se ve que la renta de bicicletas es mas importante en los días laborales y fines de semana no festivos, algo un poco paradójico, pero que se sustenta desde el hecho de que este servicio es altamente utilizado en las horas pico (6-9 am) y (4-7 pm), que está muy relacionado con los días laborales y para el resto de horas las rentas se comportan superiores para los días festivos y fines de semana que está relacionado con horas del día en la que las personas salen a pasear o a realizar actividades lúdicas. 